# Imports

In [812]:
import pandas as pd
import os
import string
from itertools import groupby, product, combinations
from heapq import nlargest
from functools import lru_cache
import re
import numpy as np
from copy import deepcopy
import pprint
pp = pprint.PrettyPrinter(indent=2)
import networkx as nx
from collections import defaultdict, deque
from datetime import datetime
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [813]:
def open_source(day):
    with open(f'source/day{day}.txt', 'r') as f:
        lista = f.readlines()
    lista = [x.strip('\n') for x in lista]
    return lista

In [814]:
path = '/Users/barnabas.kadar/Own_Repos/advent/2022/source'

def create_txt_files(path):
    files = [f"day{x}" if x > 9 else f"day0{x}" for x in range(1, 26)]
    for file in files:
        try:
            with open(os.path.join(path, file + ".txt"), 'x') as fp:
                pass
            with open(os.path.join(path, file + "_example.txt"), 'x') as fp:
                pass
        except FileExistsError:
            print("file existed but continued loop")
            continue

# Day 1

In [171]:
lista = open_source("01")

In [183]:
result = [sum([int(x) for x in a[1]]) for a in groupby(lista, key=lambda s: not s) if not a[0]]

In [184]:
max(result)

71780

In [187]:
sum(nlargest(3, result))

212489

# Day 2

In [71]:
lista = open_source("02")

In [72]:
scoring1 = {
    'A X': 4,
    'A Y': 8,
    'A Z': 3,
    'B X': 1,
    'B Y': 5,
    'B Z': 9,
    'C X': 7,
    'C Y': 2,
    'C Z': 6
}

In [73]:
scoring2 = {
    'A X': 3,
    'A Y': 4,
    'A Z': 8,
    'B X': 1,
    'B Y': 5,
    'B Z': 9,
    'C X': 2,
    'C Y': 6,
    'C Z': 7
}

In [74]:
sum([scoring1[x] for x in lista])

10624

In [75]:
sum([scoring2[x] for x in lista])

14060

# Day 3

In [4]:
lista = open_source("03")

In [5]:
scoring = dict(zip(list(string.ascii_lowercase) + list(string.ascii_uppercase), list(range(1, 53))))

In [26]:
total = 0
for sack in lista:
    half = len(sack)//2
    first, second = set(sack[half:]), set(sack[:half])
    inter = set.intersection(first, second).pop()
    total += scoring[inter]
total

7821

In [7]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [25]:
total = 0
for sack in chunks(lista, 3):
    inter = set.intersection(*list(map(lambda x: set(x), sack)))
    total += scoring[inter.pop()]
total

2752

# Day 4

In [5]:
lista = open_source('04')

In [10]:
count1 = count2 = 0
for task in lista:
    fs, fe, ss, se = list(map(int, re.split(',|-', task)))
    if (fs >= ss and fe <= se) or (fs <= ss and fe >= se):
        count1 += 1
    if fs <= se and ss <= fe:
        count2 += 1
print(count1, count2)

513 878


# Day 5

In [438]:
lista = open_source('05')

In [439]:
def convert_day5_input(lista):
    stacks = []
    proc = []
    stacks_dict = {}
    for sor in lista:
        if len(sor) > 20:
            stacks.append(list(sor[1::4]))
        elif 'move' in sor:
            proc.append(re.findall('\d+', sor))
    stacks = np.rot90(stacks, 3)
    for stack in stacks:
        k, *v = [x for x in stack if x != ' ']
        stacks_dict[k] = v
    return stacks_dict, proc

def get_day5(s, proc, part_1=True):
    stacks = deepcopy(s)
    for qty, old, new in proc:
        qty = int(qty)
        if part_1:
            for _ in range(qty):
                stacks[new].append(stacks[old].pop(-1))
        else:
            stacks[new] += stacks[old][-qty:]
            del stacks[old][-qty:]
    return ''.join([x[-1] for x in stacks.values()])

In [440]:
s, p = convert_day5_input(lista)

In [441]:
get_day5(s, p, part_1=True)

'TPGVQPFDH'

In [442]:
get_day5(s, p, part_1=False)

'DMRDFRHHH'

# Day 6

In [58]:
lista = open_source('06')

In [61]:
buffer = 14
for elem in lista:
    for i, c in enumerate(elem):
        result = elem[i:i + buffer]
        if len(set(result)) == buffer:
            print(i + buffer)
            break

3534


# Day 7

In [128]:
lista = open_source('07')

In [146]:
path = []
files = defaultdict(int)
for l in lista:
    if l.startswith('$'):
        if ' cd ' in l:
            if l.endswith('..'):
                path.pop()
            else:
                path.append(l.split()[-1])
    elif not l.startswith('dir'):
        for i, _ in enumerate(path):
            files['|'.join(path[:i + 1])] += int(l.split()[0])
            
sum(x for x in files.values() if x <= 100000)

1444896

In [147]:
min(x for x in files.values() if x >= files['/'] - 40000000)

404395

# Day 8

In [5]:
lista = open_source('08')
lista = np.array([[int(x) for x in list(x)] for x in lista])

In [15]:
def visibl(part):
    return all(np.greater(C, part))

def view(part, reverse=False):
    if reverse:
        part = np.flip(part)
    result = np.argmax(np.append(part, 9) >= C)
    if len(part) != result:
        result += 1
    return result

In [20]:
visible = 4 * (len(lista) - 1)
max_scenic = 0
R = range(1, len(lista) - 1)
for x, y in product(R, R):
    C, l, u, r, d = lista[x, y], lista[x, :y], lista[:x, y], lista[x, y + 1:], lista[x + 1:, y]
    
    if visibl(l) or visibl(u) or visibl(r) or visibl(d):
        visible += 1
        
    scenic = view(l, True) * view(u, True) * view(r) * view(d)
    if max_scenic < scenic:
        max_scenic = scenic
print(f"part1: {visible}\npart2: {max_res}")

part1: 1851
part2: 574080


# Day 9

In [664]:
lista = open_source('09')

In [665]:
dirs = dict(zip(['R', 'U', 'L', 'D'], [(1, 0), (0, 1), (-1, 0), (0, -1)]))

def which_dir(h, t):
    result = []
    if h[0] == t[0] or h[1] == t[1]:
        if h[0] - t[0] > 0:
            result.append('R')
        elif h[0] - t[0] < 0:
            result.append('L')
        elif h[1] - t[1] > 0:
            result.append('U')
        elif h[1] - t[1] < 0:
            result.append('D')
    else:
        result.append('R') if h[0] > t[0] else result.append('L')
        result.append('U') if h[1] > t[1] else result.append('D')
    return result        
        
def get_prod(h):
    return list(product((h[0]-1, h[0], h[0]+1), (h[1]-1, h[1], h[1]+1)))

In [668]:
visited = {(0, 0)}
length = 10
snake = [(0, 0)] * length
for l in lista:
    d, n = l.split()
    
    for _ in range(int(n)):
        snake[0] = tuple(np.add(snake[0], dirs[d]))
        for h, t in zip(range(0, len(snake)), range(1, len(snake))):
            if snake[t] not in get_prod(snake[h]):
                for w in which_dir(snake[h], snake[t]):
                    snake[t] = tuple(np.add(snake[t], dirs[w]))
            if t == length - 1:
                visited.add(snake[t])

In [669]:
len(visited)

2651

# Day 10

In [275]:
source = open_source('10')
lista = []
for l in source:
    if l.startswith('a'):
        lista.append('noop')
        lista.append(l)
    else:
        lista.append('noop')

In [276]:
strength, cycle, signal_sum, pixel = 1, 0, 0, ''
for l in lista:
    if cycle % 40 - 1 <= strength <= cycle % 40 + 1:
        pixel += "#"
    else:
        pixel += "."
    if cycle + 1 in list(range(20, 221, 40)):
        signal_sum += strength * (cycle + 1)
    if l.startswith('n'):
        cycle += 1
    elif l.startswith('a'):
        strength += int(l.split()[-1])
        cycle += 1
print(signal_sum)
_ = [print(pixel[40 * x:40 * x + 40]) for x in range(6)]

13860
###..####.#..#.####..##....##..##..###..
#..#....#.#..#.#....#..#....#.#..#.#..#.
#..#...#..####.###..#.......#.#....###..
###...#...#..#.#....#.##....#.#....#..#.
#.#..#....#..#.#....#..#.#..#.#..#.#..#.
#..#.####.#..#.#.....###..##...##..###..


# Day 11

In [281]:
source = open_source('11')
source.append('')

lista = defaultdict(lambda: defaultdict(str))
item_status = defaultdict(list)
total_modulo = 1

for monkey, start, op, test, true, false, _ in zip(*[iter(source)]*7):
    d = defaultdict(str)
    monke = monkey.lower().rstrip(':')
    item_status[monke]= list(map(int, start.split(': ')[-1].split(', ')))
    d['op'] = op.split(' = ')[-1]
    d['test'] = test.split(': ')[-1].replace('divisible by ', 'new % ') + ' == 0'
    total_modulo *= int(test.split()[-1])
    d['true'] = true.split('to ')[-1]
    d['false'] = false.split('to ')[-1]
    lista[monke] = d

In [282]:
inspections = defaultdict(int)
for r in range(10000):
    for monkey, a in lista.items():
        for old in item_status[monkey]:
            new = eval(a['op']) % total_modulo # // 3
            if eval(a['test']):
                item_status[a['true']].append(new)
            else:
                item_status[a['false']].append(new)
            inspections[monkey] += 1
        item_status[monkey].clear()

In [283]:
order = sorted(list(inspections.values()))
order[-2] * order[-1]

14106266886

# Day 12

In [1029]:
source = open_source('12')
source = np.array([[ord(c) - ord('a') for c in list(x)] for x in source])
sx, sy = list(zip(*np.where(source == -14)))[0]
ex, ey = list(zip(*np.where(source == -28)))[0]
source[source==-14] = 0
source[source==-28] = 25

directions = [(-1, 0), (0, -1), (1, 0), (0, 1)]
lenx, leny = source.shape[0], source.shape[1]

In [1030]:
def get_shortest_hike(queue):
    status = defaultdict(int)
    for x, y in queue:
        status[x, y] = 0
    result = []
    num = 0
    while queue:
        cx, cy = queue.popleft()
        num += 1
        if (cx, cy) == (ex, ey) or num > 50000:
            result = status[ex, ey]
            break
        for dx, dy in directions:
            nx, ny = cx + dx, cy + dy
            if (
                nx in range(lenx) and
                ny in range(leny) and
                (nx, ny) not in status
                and (source[cx, cy] - source[nx, ny]) > -2
            ):
                queue.append((nx, ny))
                status[nx, ny] = status[cx, cy] + 1
    return result

In [1031]:
queue = deque([(sx, sy)])
get_shortest_hike(queue)

449

In [1032]:
result = []
for x, y in list(zip(*np.where(source == 0))):
    queue = deque([(x, y)])
    sub_result = get_shortest_hike(queue)
    if sub_result:
        result.append(sub_result)
min(result)

443

# Day 13

# Day 14

# Day 15

# Day 16

# Day 17

# Day 18

# Day 19

# Day 20

# Day 21

# Day 22

# Day 23

# Day 24

# Day 25